In [3]:
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
os.environ["OPEN_AI_KEY"] = os.getenv("OPEN_AI_KEY")
os.environ["OPENAI_API_VERSION"] = os.getenv("OPENAI_API_VERSION")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [5]:
import os
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(model="gpt-4o", api_key=os.getenv("OPEN_AI_KEY"), api_version=os.getenv("OPENAI_API_VERSION"))


In [6]:
result = llm.invoke("What is Google")

print(result.content)

Google LLC is an American multinational technology company specializing in various internet-related services and products. These include online advertising technologies, a search engine, cloud computing, software, and hardware. It was founded in September 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University in California.

Google is best known for its search engine, which handles billions of searches per day and is the most widely used web-based search engine globally. Besides its search engine, Google offers a vast array of products and services, including:

- **Gmail**: An email service.
- **Google Maps**: A mapping and navigation service.
- **Google Drive**: A cloud storage service.
- **YouTube**: A video-sharing platform.
- **Android**: An operating system for mobile devices.
- **Google Chrome**: A web browser.
- **Google Photos**: A photo-storage and sharing service.
- **Google Assistant**: A virtual assistant powered by artificial intelligence.

In [7]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# prompt=ChatPromptTemplate.from_messages([
#     ("system", "you are customer support from doko moko you need to asnwer user queries"),
#     ("user", "{input}")
# ])
# prompt

output_parser  = JsonOutputParser()
# prompt = PromptTemplate(
#     template="Answer the user query. \n{format_instructions}\n{query}",
#     input_variables=["query"],
#     partial_variables={"format_instructions":{output_parser.get_format_instructions()}}
# )

In [8]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://github.com/RajuGangitla/nueral-backend")
loader


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
document = loader.load()

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitting = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

documents = text_splitting.split_documents(document)

In [11]:
import os
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
        azure_deployment="text-embedding-ada-002",
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("OPEN_AI_KEY"),
        api_version="2023-05-15"
        )

In [12]:
from langchain_community.vectorstores import FAISS

vectorstore  = FAISS.from_documents(documents, embeddings)
vectorstore

In [13]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    Answer the follwing questions from the provided context : 
    <content>
    {context}
    </content>
    """
)


In [14]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)


In [17]:
from langchain_community.chat_models import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_messages(
    [("system", "What are everyone's favorite colors:\n\n{context}")]
)
chain = create_stuff_documents_chain(llm, prompt)

docs = [
    Document(page_content="Jesse loves red but not yellow"),
    Document(page_content = "Jamal loves green but not as much as he loves orange")
]


In [18]:
# define schema
from pydantic import BaseModel, Field
class ResponseFormatter(BaseModel):
    """Always use this tool to structure your response to the user."""
    answer: str = Field(description="The answer to the user's question")
    followup_question: str = Field(description="A followup question the user could ask")


structered_chain = chain.with_structured_output(ResponseFormatter)   

AttributeError: 'RunnableSequence' object has no attribute 'with_structured_output'